<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/notebook/gradio_with_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chat凉宫春日的Gradio图片版本

[![Code License](https://img.shields.io/badge/Code%20License-Apache_2.0-green.svg)]()
[![Data License](https://img.shields.io/badge/Data%20License-CC%20By%20NC%204.0-red.svg)]()

**Chat凉宫春日**是模仿凉宫春日等一系列动漫人物，使用近似语气、个性和剧情聊天的语言模型，

<details>
  <summary> 本项目由李鲁鲁，冷子昂，闫晨曦，封小洋，scixing等开发。 </summary>

李鲁鲁发起了项目，并完成了最早的版本，在多个微信群实现了测试。完成了GPT自动生成对话部分。

冷子昂参与了早期Gradio的开发，并且参与了后端和前端的选型。debug和最终上线了app.py

闫晨曦将李鲁鲁的notebook重构为app.py

封小洋进行了中文转日文模型的选型，完成了针对台词抽取图片的工具，（将要完成）和haruhi图片分类器

scixing实践了VITS语音，完成了台词对应的语音抽取，（将要完成）特定人物的语音分类器。

贾曜恺正在实验一个带图片和语音的前端实现方案。

</details>

这个脚本是第一个图文版本的Chat凉宫春日，

使用了闫晨曦重构的app.py， 冷子昂完成了debug并完成了图文的Gradio实现

- [ ] 图片数据还在逐步增加中

运行这个notebook你需要OpenAI的API Token

项目链接 [https://github.com/LC1332/Prophet-Andrew-Ng](https://github.com/LC1332/Prophet-Andrew-Ng)

骆驼先知是[Luotuo(骆驼)](https://github.com/LC1332/Luotuo-Chinese-LLM)的子项目之一，后者由李鲁鲁，冷子昂，陈启源发起。

In [ ]:
!pip -q install transformers
!pip -q install openai
!pip -q install tiktoken
!pip -q install langchain
!pip -q install gradio

In [ ]:
!git clone https://github.com/LC1332/Chat-Haruhi-Suzumiya.git

Cloning into 'Chat-Haruhi-Suzumiya'...
remote: Enumerating objects: 402, done.
remote: Counting objects: 100% (176/176), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 402 (delta 75), reused 126 (delta 41), pack-reused 226
Receiving objects: 100% (402/402), 10.52 MiB | 15.17 MiB/s, done.
Resolving deltas: 100% (154/154), done.


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = 'sk-DfFyRKOnKSZ'

In [ ]:
%cd Chat-Haruhi-Suzumiya/src

/content/Chat-Haruhi-Suzumiya/src


In [ ]:
!python app.py

A new version of the following files was downloaded from https://huggingface.co/silk-road/luotuo-bert:
- models.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
文件夹 'Suzumiya' 创建成功！
/content/Chat-Haruhi-Suzumiya/src/app.py:323: UserWarning: You have unused kwarg parameters in Textbox, please remove them: {'placeholde': '输入角色名'}
  role_name = gr.Textbox(label="角色名", placeholde="输入角色名")
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://ceb568aac6f653a2c6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
history done
2
3
['不重要的事情', '为什么剪头发', '自我介绍', '介绍其他社员', '开学第二天', '电脑是怎么来的', 'SOS团起名由来']
当前辅助sample: ['不重要的事情', '为什么剪头发', '自我介绍', '介绍其他社员', '开学第二天', '电脑是怎么来的', 'SOS团起名由来']
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=854x48